In [1]:
import pandas as pd
from statistics import median

In [2]:
ci_skip = pd.read_csv('ci_skipped_dynamic_variants.csv')
base = pd.read_csv('baseline_dynamic_batching.csv')

In [3]:
projects = list(set(ci_skip['project'].tolist()))

In [4]:
algorithms = ['BATCHBISECT', 'BATCHSTOP4', 'BATCHDIVIDE4']
types = set(ci_skip['update_method'].tolist())

In [5]:
db = {}

In [6]:
min_max_db = {}

In [9]:
for alg in algorithms:
    print(alg)
    db[alg] = {}
    min_max_db[alg] = {}
    
    skip_alg = ci_skip[ ci_skip['algorithm'] == alg]
    base_alg = base[ base['algorithm'] == alg]
    
#     print(base_alg)
    
    builds = base_alg['builds_saved'].tolist()
    projects = base_alg['project'].tolist()
    
    for i in range(len(projects)):
        db[alg][projects[i]] = [builds[i], 0]
        
    for t in types:
        print(t)
        skip_alg_t = skip_alg[ skip_alg['update_method']==t]
        skip_alg_t = skip_alg_t.sort_values(by=['project'])
        
#         print(skip_alg_t)
        
        builds = skip_alg_t['builds_saved'].tolist()
        projects = skip_alg_t['project'].tolist()
        
        for i in range(len(projects)):
            db[alg][projects[i]][1] = builds[i]
        
        l = []
        
        for p in db[alg]:
            l.append(abs(db[alg][p][0]-db[alg][p][1]))
        
        projects = list(db[alg].keys())
        
        print('Min: {} {}'.format(min(l), projects[l.index(min(l))]))
        print('Max: {} {}'.format(max(l), projects[l.index(max(l))]))
        min_max_db[alg][t] = [(min(l), projects[l.index(min(l))]), (max(l), projects[l.index(max(l))])]
        
#         print(db[alg])

BATCHBISECT
linear
Min: 0.009102033798868092 dropwizard-dropwizard.csv
Max: 48.05294751423735 essentials-Essentials.csv
exponential
Min: 0.00829084276416836 caelum-vraptor4.csv
Max: 17.915961212867476 essentials-Essentials.csv
random_linear
Min: 0.007916748886628966 yegor256-takes.csv
Max: 15.176235185470226 essentials-Essentials.csv
random_exponential
Min: 0.04947712440299057 xetorthio-jedis.csv
Max: 9.835336003560311 apache-pdfbox.csv
stagger
Min: 0.16187188339086944 linkedin-gobblin.csv
Max: 11.066646144374339 essentials-Essentials.csv
BATCHSTOP4
linear
Min: 0.24737520074210817 apache-sling.csv
Max: 11.44674997134014 cloudfoundry-uaa.csv
exponential
Min: 0.17021470691494311 apache-sling.csv
Max: 10.041259037114406 grails-grails-core.csv
random_linear
Min: 0.24036061039418655 apache-sling.csv
Max: 10.220101585347095 essentials-Essentials.csv
random_exponential
Min: 0.06499585169609201 apache-sling.csv
Max: 10.220101585347095 essentials-Essentials.csv
stagger
Min: 0.18424388761078614 

In [10]:
for alg in db:
    keys = list(db[alg].keys())
    vals = list(db[alg].values())
    
    print('For {}, \nMinimum {} = {}, \nMaximum = {} = {}'.format(alg, keys[vals.index(min(vals))], min(vals), keys[vals.index(max(vals))], max(vals)))
    

For BATCHBISECT, 
Minimum essentials-Essentials.csv = [-6.741573033707859, -17.808219178082197], 
Maximum = checkstyle-checkstyle.csv = [77.31211090430654, 81.58306838134105]
For BATCHSTOP4, 
Minimum essentials-Essentials.csv = [-3.370786516853941, 4.109589041095887], 
Maximum = checkstyle-checkstyle.csv = [78.49089703271078, 83.17515303943435]
For BATCHDIVIDE4, 
Minimum essentials-Essentials.csv = [-3.370786516853941, -5.479452054794521], 
Maximum = checkstyle-checkstyle.csv = [78.55079905012515, 66.2553979025293]


In [10]:
vals.index(min(vals))

3

In [11]:
keys[3]

'apache-sling.csv-stagger-linear'